In [36]:
import fitz
import tner
import pandas as pd
tner_tagger = tner.TransformersNER('asahi417/tner-xlm-roberta-base-ontonotes5')

2022-03-16 12:40:03 INFO     *** initialize network ***


In [34]:
def get_sentences(document_fname):
    document = fitz.open(document_fname)
    sentences = []
    for page in document:
        page_text = page.get_text()
        page_sentences = page_text.split("\n")
        page_sentences = [sentence.strip() for sentence in page_sentences]
        sentences.extend(page_sentences)
    sentences = [sentence.strip() for sentence in sentences if len(sentence.strip())>0]
    return sentences
fname = "../PDF/Eurofins 01.pdf"
get_sentences(fname)

['1048318294708',
 'Eurofins COVID Testing Services Limited',
 'Queens Road,',
 'Teddington,',
 'Middlesex,',
 'TW11 0LY,',
 'UK',
 'Customer Services +44 (0)1925 980 595, covid19administration@eurofins.co.uk',
 'Kai Lukas Lau',
 'Travel testing customer',
 '02 March 2022',
 'Barcode of sample: COV1105006ACONLFDAG00337815',
 'Test Report SARS-CoV-2 (COVID-19)',
 'PATIENT INFORMATION',
 'Last name:',
 'Lau',
 'First name:',
 'Kai Lukas',
 'Birth date:',
 '2011-10-07',
 'Passport:',
 'K2217817Z',
 'Gender:',
 'Male',
 'Nationality:',
 'Singapore',
 'Application ID:',
 '4152458',
 'HRID/Badge ID:',
 '2202871023cxq',
 'Sample kit ID:',
 'COV1105006ACONLFDAG00337815',
 'Date and time of',
 'sampling:',
 '2022-02-08 17:02',
 'Sample received in the',
 'laboratory:',
 'Date and time of result',
 'reporting:',
 'Yes',
 '2022-02-08 17:26',
 'Date and time of report',
 'generation:',
 '2022-03-02 11:26',
 'TEST',
 'RESULT',
 'REFERENCE',
 'METHOD',
 'Fit to Fly Antigen Testing -',
 'Drive Throug

In [47]:
def get_entities(list_of_strings):
    sentence_id, category, start_pos, end_pos, text, confidence_score = [], [], [], [], [], []
    for id, sentence in enumerate(get_sentences(fname)):
        result = tner_tagger.predict([sentence])[0]['entity']
        if len(result)>0:
            for item in result:
                sentence_id.append(id)
                category.append(item['type'])
                start_pos.append(item['position'][0])
                end_pos.append(item['position'][1])
                text.append(item['mention'])
                confidence_score.append(item['probability'])
    entities = pd.DataFrame({"SentenceID": sentence_id, "EntityType": category, "StartPosition": start_pos, "EndPosition": end_pos, "EntityText": text, "ConfidenceScore": confidence_score})
    entities = entities[entities.EntityType.isin(["person", "date", "time"])]
    # entities = entities.groupby('SentenceID').agg(",".join).reset_index()
    return entities
entities = get_entities(get_sentences(fname))
entities

,SentenceID,EntityType,StartPosition,EndPosition,EntityText,ConfidenceScore
6,8,person,0,13,Kai Lukas Lau,0.982803
7,10,date,0,13,02 March 2022,0.773422
8,15,person,0,3,Lau,0.851363
10,17,person,0,9,Kai Lukas,0.988147
12,19,date,0,10,2011-10-07,0.647136
14,34,date,0,10,2022-02-08,0.738792
15,34,time,11,16,17:02,0.860903
16,40,date,0,10,2022-02-08,0.752259
17,40,time,11,16,17:26,0.866541
18,43,date,0,10,2022-03-02,0.708695
